# Setting up kaggle to download dataset

We're gonna be using the anime face dataset from Kaggle. https://www.kaggle.com/datasets/splcher/animefacedataset

This contains 63000 images of anime characters. Visit the website for more information.

In [1]:
!pip install -q kaggle

In [2]:
! mkdir ~/.kaggle

In [10]:
! cp kaggle.json ~/.kaggle/

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download splcher/animefacedataset

 99% 392M/395M [00:20<00:00, 21.9MB/s]
100% 395M/395M [00:20<00:00, 20.3MB/s]


In [13]:
!mkdir ./animefacedataset

mkdir: cannot create directory ‘./animefacedataset’: File exists


In [14]:
!unzip animefacedataset.zip -d ./animefacedataset

Streaming output truncated to the last 5000 lines.
  inflating: ./animefacedataset/images/62764_2019.jpg  
  inflating: ./animefacedataset/images/62765_2019.jpg  
  inflating: ./animefacedataset/images/62766_2019.jpg  
  inflating: ./animefacedataset/images/62767_2019.jpg  
  inflating: ./animefacedataset/images/62768_2019.jpg  
  inflating: ./animefacedataset/images/62769_2019.jpg  
  inflating: ./animefacedataset/images/6276_2003.jpg  
  inflating: ./animefacedataset/images/62770_2019.jpg  
  inflating: ./animefacedataset/images/62771_2019.jpg  
  inflating: ./animefacedataset/images/62772_2019.jpg  
  inflating: ./animefacedataset/images/62773_2019.jpg  
  inflating: ./animefacedataset/images/62774_2019.jpg  
  inflating: ./animefacedataset/images/62775_2019.jpg  
  inflating: ./animefacedataset/images/62776_2019.jpg  
  inflating: ./animefacedataset/images/62777_2019.jpg  
  inflating: ./animefacedataset/images/62778_2019.jpg  
  inflating: ./animefacedataset/images/62779_2019.jpg 

# Checking out if our dataset is correctly downloaded

In [15]:
project_name = '06b-anime-dcgan'

import os

DATA_DIR = './animefacedataset'
print(os.listdir(DATA_DIR))

['images']


In [16]:
print(os.listdir(DATA_DIR+'/images')[:10])

['13988_2006.jpg', '47837_2014.jpg', '5860_2003.jpg', '14126_2006.jpg', '59706_2018.jpg', '58084_2017.jpg', '279_2000.jpg', '16817_2007.jpg', '8841_2004.jpg', '18323_2007.jpg']


# Creating a PyTorch Dataset and DataLoader

In [17]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
torch.set_default_device(device)

In [18]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import v2 as transforms

In [19]:
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [20]:
# Creating a transform for our images
img_transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.PILToTensor(),
    transforms.ToDtype(torch.float32, scale=True),
    transforms.Normalize(*stats)  # Normalize with mean and std 0.5 for each channel
])

In [21]:
train_dataset = ImageFolder(root=DATA_DIR,
                            transform=img_transform)

train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=os.cpu_count(), pin_memory=True)

# Creating helper functions to denormalize images and visualize images from a training batch.

In [22]:
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
def denorm(img_tensors):
  return img_tensors * stats[0][1] + stats[0][0]

In [24]:
def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8).permute(1, 2, 0))

def show_batch(dl, nmax=64):
    for images, _ in dl:
        show_images(images, nmax)
        break

In [25]:
show_batch(train_dataloader)

RuntimeError: Expected a 'cuda' device type for generator but found 'cpu'

# Creating the discriminator

In [26]:
import torch.nn as nn

discriminator = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid()
    )

In [27]:
# Check which device our model is on.
next(discriminator.parameters()).device

device(type='cuda', index=0)

In [28]:
# Move model to target device
discriminator = discriminator.to(device)